In [11]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
import requests
from io import BytesIO


In [12]:

# Load pre-trained ResNet-18 model
model = models.resnet18(pretrained=True)
model.eval()
# Append a softmax layer to the model
#model = nn.Sequential(model, nn.Softmax(dim=0)) ##!not working!!
torch.save(model.state_dict(), 'zoo/resnet18_model.pth')


In [13]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [14]:


# Example input to the model
example_input = torch.randn(1, 3, 224, 224)

# Export the model to ONNX format
onnx_path = "onnx/resnet18.onnx"
torch.onnx.export(model, example_input, onnx_path, verbose=True)

print(f"Model exported to {onnx_path}")


Exported graph: graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=0, device=cpu),
      %fc.weight : Float(1000, 512, strides=[512, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %onnx::Conv_193 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %onnx::Conv_194 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_196 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_197 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_199 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_200 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_202 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %onnx::Conv_203 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %onnx::Conv_205 : Fl

In [15]:

# Perform inference on an example image
image_url = 'https://www.hartz.com/wp-content/uploads/2022/04/small-dog-owners-1.jpg'  # Provide an image URL
response = requests.get(image_url,verify=False)
img = Image.open(BytesIO(response.content))


/home/uidl7286/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [16]:
# Preprocess the image
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_data = preprocess(img)
input_data = input_data.unsqueeze(0)  


In [17]:

# Make the prediction
with torch.no_grad():
    output = model(input_data)

# Post-processing
#probabilities = output[0]
probabilities = torch.nn.functional.softmax(output[0], dim=0)
predicted_class = torch.argmax(probabilities).item()




In [18]:
# Download ImageNet labels
LABELS_URL = 'https://raw.githubusercontent.com/anishathalye/imagenet-simple-labels/master/imagenet-simple-labels.json'
labels = requests.get(LABELS_URL,verify=False).json()


/home/uidl7286/.local/lib/python3.10/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '127.0.0.1'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [19]:
import json
# Save the list to a JSON file
file_path = 'zoo/imagenet-simple-labels.json'
with open(file_path, 'w') as file:
    json.dump(labels, file)

In [20]:

# Get the predicted label
predicted_label = labels[predicted_class]

print(f"Predicted class: {predicted_class}")
print(f"Predicted label: {predicted_label}")
print(f"Probability: {probabilities[predicted_class]:.2%}")

Predicted class: 187
Predicted label: Yorkshire Terrier
Probability: 98.82%
